In [2]:
from qiskit.opflow import Z, I
from qiskit.opflow import StateFn
import numpy as np
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit.circuit import QuantumCircuit
from qiskit.opflow import Z, X
from qiskit.opflow import CircuitSampler, PauliExpectation
from qiskit.opflow import PauliExpectation, CircuitSampler
from qiskit.opflow import Gradient
from qiskit.algorithms.optimizers import GradientDescent, ADAM
from qiskit.circuit.library import RealAmplitudes

In [21]:
# initial_point = np.random.random(ansatz.num_parameters)
initial_point = np.array([0.43253681, 0.09507794, 0.42805949, 0.34210341])
hamiltonian = Z ^ Z
ansatz = RealAmplitudes(num_qubits=2, reps=1, entanglement='linear')
ansatz.draw(style='iqx')

┌──────────────────────────────────────┐
q_0: ┤0                                     ├
     │  RealAmplitudes(θ[0],θ[1],θ[2],θ[3]) │
q_1: ┤1                                     ├
     └──────────────────────────────────────┘

In [22]:
backend = Aer.get_backend('qasm_simulator')
q_instance = QuantumInstance(backend, shots = 8192, seed_simulator = 2718, seed_transpiler = 2718)

In [23]:
#expectation = StateFn(hamiltonian, is_measurement=True) @ StateFn(ansatz)
#exp_result = expectation.eval()

def evaluate_expectation(x):
    value_dict = dict(zip(ansatz.parameters, x))
    bind_ansatz = ansatz.bind_parameters(value_dict)
    
    expectation = StateFn(hamiltonian, is_measurement=True) @ StateFn(bind_ansatz)
    result = expectation.eval()
    
    return np.real(result)

evaluate_expectation(initial_point)

0.8852483900774777

In [24]:
def evaluate_gradient(x):
    value_dict = dict(zip(ansatz.parameters, x))
    
    expectation = StateFn(hamiltonian, is_measurement=True) @ StateFn(ansatz)
    gradient = Gradient().convert(expectation).assign_parameters(value_dict)
    
    result = gradient.eval() 
    return np.real(result)   

evaluate_gradient(initial_point)

array([ 0.13857597, -0.35720615, -0.24934966, -0.21377437])

In [35]:
gd_loss = []
def gd_callback(nfevs, x, fx, stepsize):
    gd_loss.append(fx)
    
gd = GradientDescent(maxiter=300, learning_rate=0.01, callback=gd_callback)

In [36]:
x_opt, fx_opt, nfevs = gd.optimize(initial_point.size,    # number of parameters
                                   evaluate_expectation,  # function to minimize
                                   gradient_function=evaluate_gradient,  # function to evaluate the gradient
                                   initial_point=initial_point)  # initial point

In [39]:
print(x_opt, fx_opt, nfevs)

[-0.31391274  1.47656398  0.33825704  1.60692521] -0.9982793854608492 300


In [40]:
help(gd.optimize)

Help on method optimize in module qiskit.algorithms.optimizers.gradient_descent:

optimize(num_vars, objective_function, gradient_function=None, variable_bounds=None, initial_point=None) method of qiskit.algorithms.optimizers.gradient_descent.GradientDescent instance
    Perform optimization.
    
    Args:
        num_vars (int) : Number of parameters to be optimized.
        objective_function (callable) : A function that
            computes the objective function.
        gradient_function (callable) : A function that
            computes the gradient of the objective function, or
            None if not available.
        variable_bounds (list[(float, float)]) : List of variable
            bounds, given as pairs (lower, upper). None means
            unbounded.
        initial_point (numpy.ndarray[float]) : Initial point.
    
    Returns:
        point, value, nfev
           point: is a 1D numpy.ndarray[float] containing the solution
           value: is a float with the object

In [18]:
import numpy as np
a = np.array([1,2])
np.tile(a,(5,1))

array([[1, 2],
       [1, 2],
       [1, 2],
       [1, 2],
       [1, 2]])

In [8]:
num_inputs = 5
num_params = 7
param_dim = 12
input_dim = 2

rep_range = np.tile(np.array([num_inputs]), num_params)
params = np.random.uniform(0, 2*np.pi, size=(num_params, param_dim))
grid_params = np.repeat(params, repeats=rep_range, axis=0)
inputs = np.random.normal(0, 1, size=(num_inputs, input_dim))
grid_inputs = np.tile(inputs, (num_params, 1))

In [10]:
params.shape

(7, 12)

In [11]:
inputs.shape

(5, 2)

In [3]:
help(ADAM)

Help on class ADAM in module qiskit.algorithms.optimizers.adam_amsgrad:

class ADAM(qiskit.algorithms.optimizers.optimizer.Optimizer)
 |  ADAM(maxiter: int = 10000, tol: float = 1e-06, lr: float = 0.001, beta_1: float = 0.9, beta_2: float = 0.99, noise_factor: float = 1e-08, eps: float = 1e-10, amsgrad: bool = False, snapshot_dir: Union[str, NoneType] = None) -> None
 |  
 |  Adam and AMSGRAD optimizers.
 |  
 |  Adam [1] is a gradient-based optimization algorithm that is relies on adaptive estimates of
 |  lower-order moments. The algorithm requires little memory and is invariant to diagonal
 |  rescaling of the gradients. Furthermore, it is able to cope with non-stationary objective
 |  functions and noisy and/or sparse gradients.
 |  
 |  AMSGRAD [2] (a variant of Adam) uses a 'long-term memory' of past gradients and, thereby,
 |  improves convergence properties.
 |  
 |  References:
 |  
 |      [1]: Kingma, Diederik & Ba, Jimmy (2014), Adam: A Method for Stochastic Optimization.
 

In [23]:
help(ADAM)

Help on class ADAM in module qiskit.algorithms.optimizers.adam_amsgrad:

class ADAM(qiskit.algorithms.optimizers.optimizer.Optimizer)
 |  ADAM(maxiter: int = 10000, tol: float = 1e-06, lr: float = 0.001, beta_1: float = 0.9, beta_2: float = 0.99, noise_factor: float = 1e-08, eps: float = 1e-10, amsgrad: bool = False, snapshot_dir: Union[str, NoneType] = None) -> None
 |  
 |  Adam and AMSGRAD optimizers.
 |  
 |  Adam [1] is a gradient-based optimization algorithm that is relies on adaptive estimates of
 |  lower-order moments. The algorithm requires little memory and is invariant to diagonal
 |  rescaling of the gradients. Furthermore, it is able to cope with non-stationary objective
 |  functions and noisy and/or sparse gradients.
 |  
 |  AMSGRAD [2] (a variant of Adam) uses a 'long-term memory' of past gradients and, thereby,
 |  improves convergence properties.
 |  
 |  References:
 |  
 |      [1]: Kingma, Diederik & Ba, Jimmy (2014), Adam: A Method for Stochastic Optimization.
 

In [5]:
opt = ADAM()

In [34]:
f0 = opt.wrap_function(f, y0)

In [35]:
f0(1)

2

In [37]:
raise Exception('1')

Exception: 1

In [8]:
def f(x,y):
    return x+y

In [29]:
y0 = tuple((1,))